In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vermaavi/food11")

print("Path to dataset files:", path)

Section 1: Setup and Data Loading

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import cv2
import os

# Define paths
DATASET_PATH = "/content/drive/My Drive/HSIFoodIngr-64"  # Update the path to your dataset in Drive
IMAGE_SIZE = (256, 256)
NUM_CLASSES = 64

# Function to load and preprocess hyperspectral images
def load_images_and_labels(data_path):
    images = []
    labels = []
    ingredient_mapping = {}
    current_label = 0

    for ingredient in os.listdir(data_path):
        ingredient_path = os.path.join(data_path, ingredient)
        if os.path.isdir(ingredient_path):
            ingredient_mapping[current_label] = ingredient
            for img_file in os.listdir(ingredient_path):
                img_path = os.path.join(ingredient_path, img_file)
                img = cv2.imread(img_path)
                img = cv2.resize(img, IMAGE_SIZE)
                images.append(img)
                labels.append(current_label)
            current_label += 1

    images = np.array(images) / 255.0  # Normalize images
    labels = np.array(labels)
    return images, labels, ingredient_mapping

# Load dataset
images, labels, ingredient_mapping = load_images_and_labels(DATASET_PATH)


Section 2: Train-Test Split

In [ ]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


Model Definition

In [ ]:
# Load pre-trained ResNet-50 model without the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(1024, activation='relu'),
    Dropout(0.3),
    Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0005),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


 Training the Model

In [ ]:
# Train the model
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test),
                    epochs=50, 
                    batch_size=32, 
                    verbose=1)


Evaluation and Save the Model

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Generate classification report
y_pred = np.argmax(model.predict(X_test), axis=1)
print(classification_report(y_test, y_pred, target_names=list(ingredient_mapping.values())))

# Save the model
model.save("/content/drive/My Drive/ResNet50_HSIFoodIngr64.h5")


Load the Model and Predict

In [ ]:
# Load the model for inference
loaded_model = tf.keras.models.load_model("/content/drive/My Drive/ResNet50_HSIFoodIngr64.h5")

# Predict on a new image
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, IMAGE_SIZE) / 255.0
    return np.expand_dims(img, axis=0)

new_image_path = "/content/drive/My Drive/sample_image.jpg"  # Update with your sample image path
new_image = preprocess_image(new_image_path)
prediction = np.argmax(loaded_model.predict(new_image), axis=1)
predicted_ingredient = ingredient_mapping[prediction[0]]
print(f"Predicted Ingredient: {predicted_ingredient}")
